In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import compute_class_weight
import matplotlib.pyplot as plt

In [42]:
data = []
for filename in os.listdir("Cropped/UTKFace"):
    if filename.endswith(".jpg"):
        row_data = [os.path.join("Cropped/UTKFace", filename)]
        filename = filename.split("_")
        if len(filename) > 3:
            # TODO REPLACE INT WITH NAME
            row_data.append(filename[2]) # 3rd integer represents race
            data.append(row_data)

In [43]:
df_image = pd.DataFrame(data, columns = ["filename", "Race"])

df_image = df_image[df_image["Race"] != "4"]

train, test = train_test_split(df_image, stratify = df_image["Race"], test_size=0.2)

# train = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255).flow_from_dataframe(
#     dataframe = train,
#     x_col = "filename",
#     y_col = "Race",
#     class_mode = "categorical",
#     target_size = (200, 200),
#     fill_mode = "nearest",
#     shuffle=True
# )

train = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255,
                                                        fill_mode="nearest").flow_from_dataframe(
    dataframe = train,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    target_size = (200, 200),
    shuffle=True
)

test = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255).flow_from_dataframe(
    dataframe = test,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    fill_mode = "nearest",
    target_size=(200, 200)
)
    
# batch = next(train)
# a = np.array(batch[0])

# img = Image.fromarray(a[0].astype('uint8'), "RGB")
# img.save(f"test.png")
# img.show()

Found 17610 validated image filenames belonging to 4 classes.
Found 4403 validated image filenames belonging to 4 classes.


In [44]:
df_image["Race"].value_counts(normalize=True)

Race
0    0.457820
1    0.205606
3    0.180575
2    0.155999
Name: proportion, dtype: float64

In [45]:
compute_class_weight(
    "balanced", 
    classes = np.array(["0", "1", "2", "3"]),
    y = df_image["Race"])

# compute_class_weight(
#     "balanced", 
#     classes = np.array(["0", "1"]),
#     y = df_image["Race"])
# tf.config.list_physical_devices("GPU")

array([0.54606569, 1.21591913, 1.60257717, 1.38446541])

In [46]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(200, 200, 3)))
model.add(keras.layers.Dense(512, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(256, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(128, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(4, activation = "softmax"))
model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"], optimizer=keras.optimizers.Adam())
model.fit(train,
          class_weight={0:0.54606569 , 1:1.21591913, 2:1.60257717, 3:1.38446541},
          validation_data = test,
          epochs = 20)

Epoch 1/20
551/551 [==============================] - 15s 26ms/step - loss: 4.5628 - accuracy: 0.2150 - val_loss: 1.3885 - val_accuracy: 0.2055
Epoch 2/20
551/551 [==============================] - 14s 26ms/step - loss: 1.3865 - accuracy: 0.1999 - val_loss: 1.3833 - val_accuracy: 0.4579
Epoch 3/20
551/551 [==============================] - 14s 26ms/step - loss: 1.3941 - accuracy: 0.2632 - val_loss: 1.3820 - val_accuracy: 0.4579
Epoch 4/20
551/551 [==============================] - 14s 25ms/step - loss: 1.4028 - accuracy: 0.4057 - val_loss: 1.3937 - val_accuracy: 0.1806
Epoch 5/20
551/551 [==============================] - 14s 25ms/step - loss: 1.3867 - accuracy: 0.2120 - val_loss: 1.3863 - val_accuracy: 0.1560
Epoch 6/20
551/551 [==============================] - 14s 25ms/step - loss: 1.3868 - accuracy: 0.2123 - val_loss: 1.3848 - val_accuracy: 0.4579
Epoch 7/20
551/551 [==============================] - 14s 26ms/step - loss: 1.3865 - accuracy: 0.2143 - val_loss: 1.3826 - val_accuracy:

In [47]:
model.save("mlp_model.keras")

In [49]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (5, 5), padding = "same", input_shape=(200, 200, 3), activation = "relu"))
model.add(keras.layers.Conv2D(32, (5, 5), padding = "same", activation = "relu"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.Conv2D(64, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.Conv2D(128, (3, 3), padding = "same", activation = "relu"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation = "relu"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(4, activation = "softmax"))

model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"], optimizer=keras.optimizers.Adam())
model.fit(train,
          validation_data = test,
          class_weight={0:0.54606569, 1:1.21591913 , 2:1.60257717, 3:1.38446541},
          batch_size = 64,
          epochs = 20)

Epoch 1/20
551/551 [==============================] - 69s 122ms/step - loss: 1.1742 - accuracy: 0.5225 - val_loss: 0.8590 - val_accuracy: 0.6852
Epoch 2/20
551/551 [==============================] - 64s 115ms/step - loss: 0.7716 - accuracy: 0.6463 - val_loss: 0.7035 - val_accuracy: 0.7340
Epoch 3/20
551/551 [==============================] - 59s 107ms/step - loss: 0.6601 - accuracy: 0.6918 - val_loss: 0.6391 - val_accuracy: 0.7836
Epoch 4/20
551/551 [==============================] - 58s 106ms/step - loss: 0.6187 - accuracy: 0.7073 - val_loss: 0.6253 - val_accuracy: 0.7974
Epoch 5/20
551/551 [==============================] - 58s 106ms/step - loss: 0.5531 - accuracy: 0.7357 - val_loss: 0.5417 - val_accuracy: 0.8235
Epoch 6/20
551/551 [==============================] - 58s 105ms/step - loss: 0.5057 - accuracy: 0.7612 - val_loss: 0.6760 - val_accuracy: 0.7699
Epoch 7/20
551/551 [==============================] - 58s 105ms/step - loss: 0.4633 - accuracy: 0.7870 - val_loss: 0.5220 - val_ac

In [50]:
model.save("cnn_model")

INFO:tensorflow:Assets written to: cnn_model\assets


INFO:tensorflow:Assets written to: cnn_model\assets


In [ ]:
img = Image.open(
    r"C:\Users\Allen\Downloads\IMG_0791.jpg"
)

img = img.resize((200, 200))
img_array = keras.utils.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)
predicted = model.predict(img_array)
# print(np.argmax(predicted, axis = 1)[0])
print(predicted)

1/1 [==============================] - 0s 16ms/step
[[9.9998045e-01 1.5736923e-10 1.3597108e-10 1.9519108e-05]]


In [ ]:
import tensorflow as tf
import keras as keras
model = tf.keras.models.load_model(r"C:\Users\Allen\Desktop\Coding\CSC 466\FacialRecognitionRacialBias\cnn_model")

In [ ]:
model.save("cnn_model.keras")